2024/1/2  
Mengyang Xu  
  
### Updates
In section 4, for the questions data of the QAconv data, the embedding methodology also be added to the retrieval system to see if it has positive impacts for the improvement of performance(f1). Moreover, the retrieval sentence is also being used to compare with the GPT4ALL model.  
  
### Findings
1. After adding the embedding, the performance of the system did not make a big difference, perhaps there is still error with the embedding process  
2. The retrieve sentence method did solve some prblems for the GPT4ALL model, it located some specific sentences that is related to the question while the GPT4ALL model can not answer. However, it also located some sentences that can not be used to answer the question.

## 1. Data pre-process

Combine the questions file and source text together

from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import GPT4AllEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import GPT4All
from langchain.chains import RetrievalQA
from collections import Counter
import numpy as np
from collections import defaultdict
import json
from pathlib import Path
from langchain.prompts import PromptTemplate

In [3]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import GPT4AllEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import GPT4All
from langchain.chains import RetrievalQA
from collections import Counter
import numpy as np
from collections import defaultdict
import json
from pathlib import Path
from langchain.prompts import PromptTemplate
import logging
import time
from langchain.embeddings.huggingface import HuggingFaceInstructEmbeddings
import torch

In [2]:
file_path='E:/QA/article_segment.json'
data = json.loads(Path(file_path).read_text())

In [34]:
text=''
for i in data['newsdial-984']["seg_dialog"]:
    text+=i['text']

In [36]:
for i in data['newsdial-984']['prev_ctx']:
    text+=i['text']

In [37]:
text

'And one senator, for example today, Senator Bob Casey, a Democrat from Pennsylvania, sent a letter to the president urging him - I\'m quoting from the letter - he says, "to put an end to these practices immediately, to ensure that the investment decisions being made by Freddie Mac are in line with the intent of Congress and the greater needs of our economy." So we also spoke today with Senator Johnny Isakson, a Republican from Georgia, who\'s also interested in reforming Freddie Mac and Fannie Mae. So I think there are people on Capitol Hill who are starting to really ponder, What is the structure of these organizations and what role do they play? What\'s appropriate for them to do? And this is just sort of an interesting look at conflicts of interest. We are not, in any way - our story has not said that it is illegal, just that it raises questions about how it functions.Well, you did quote Alan Boyce in your story, a former bond trader who\'s been involved in efforts to push for more

In [4]:
import pandas as pd

In [5]:
Conv = pd.read_json("E:/QA/article_segment.json")

In [6]:
Conv

,newsdial-984,newsdial-924,newsdial-599,newsdial-184,newsdial-144,newsdial-880,newsdial-983,newsdial-129,newsdial-956,newsdial-987,...,newsdial-4842,newsdial-1384,supreme_court-4434,enron_wisrlab-2432,newsdial-3004,enron_wisrlab-1707,newsdial-3646,supreme_court-1113,newsdial-3104,newsdial-3275
prev_ctx,"[{'id': 'newsidal-NPR-166-26', 'speaker': 'MAR...","[{'id': 'newsidal-NPR-162-25', 'speaker': 'NEA...","[{'id': 'newsidal-NPR-132-10', 'speaker': 'RAC...","[{'id': 'newsidal-NPR-38-10', 'speaker': 'Ms. ...","[{'id': 'newsidal-NPR-31-12', 'speaker': 'FARA...","[{'id': 'newsidal-NPR-159-100', 'speaker': 'KA...","[{'id': 'newsidal-NPR-166-23', 'speaker': 'NEA...","[{'id': 'newsidal-NPR-28-25', 'speaker': 'Ms. ...","[{'id': 'newsidal-NPR-163-145', 'speaker': 'SO...","[{'id': 'newsidal-NPR-166-35', 'speaker': 'JEF...",...,"[{'id': 'newsidal-NPR-692-3', 'speaker': 'ELIZ...","[{'id': 'newsidal-NPR-219-102', 'speaker': 'PE...","[{'id': 'court-04-698-28361', 'speaker': 'JUST...",[],"[{'id': 'newsidal-NPR-486-57', 'speaker': 'NEA...","[{'id': 'enron-enron_wisrlab_60462-0', 'speake...","[{'id': 'newsidal-NPR-554-23', 'speaker': 'I m...","[{'id': 'court-03-636-7031', 'speaker': 'JUSTI...","[{'id': 'newsidal-NPR-491-208', 'speaker': 'ST...","[{'id': 'newsidal-NPR-506-9', 'speaker': 'FARA..."
seg_dialog,"[{'id': 'newsidal-NPR-166-27', 'speaker': 'MAR...","[{'id': 'newsidal-NPR-162-26', 'speaker': 'AMY...","[{'id': 'newsidal-NPR-132-11', 'speaker': 'URI...","[{'id': 'newsidal-NPR-38-11', 'speaker': 'Ms. ...","[{'id': 'newsidal-NPR-31-13', 'speaker': 'FARA...","[{'id': 'newsidal-NPR-159-101', 'speaker': 'KA...","[{'id': 'newsidal-NPR-166-24', 'speaker': 'CHR...","[{'id': 'newsidal-NPR-28-26', 'speaker': 'MADE...","[{'id': 'newsidal-NPR-163-146', 'speaker': 'NE...","[{'id': 'newsidal-NPR-166-36', 'speaker': 'JEF...",...,"[{'id': 'newsidal-NPR-692-4', 'speaker': 'MELI...","[{'id': 'newsidal-NPR-219-103', 'speaker': 'JO...","[{'id': 'court-04-698-28362', 'speaker': 'MR. ...","[{'id': 'enron-enron_wisrlab_13011-0', 'speake...","[{'id': 'newsidal-NPR-486-58', 'speaker': 'RIC...","[{'id': 'enron-enron_wisrlab_60462-1', 'speake...","[{'id': 'newsidal-NPR-554-24', 'speaker': 'I m...","[{'id': 'court-03-636-7032', 'speaker': 'MR. D...","[{'id': 'newsidal-NPR-491-209', 'speaker': 'NE...","[{'id': 'newsidal-NPR-506-10', 'speaker': 'Dr...."
word_count,237,226,221,248,272,217,213,333,204,263,...,236,228,222,246,240,245,239,284,295,215


In [7]:
QA = pd.read_json("E:/QAConv-V1.0/QAConv-V1.0/sample.json")

In [8]:
QA['text']=''

In [9]:
QA.iloc[0]['article_segment_id']

'openslack-2635'

In [11]:
QA.loc[0,'text']='dasdawd'

In [13]:
for i in range(len(QA)):
    text=''
    for j in data[QA.iloc[i]['article_segment_id']]['prev_ctx']:
        text+=j['text']
    for k in data[QA.iloc[i]['article_segment_id']]["seg_dialog"]:
        text+=k['text']
    QA.loc[i,'text']=text

In [4]:
QA.to_json('QA_data_sample.json', orient='records')

NameError: name 'QA' is not defined

In [4]:
file_path='E:/QA/QA_data_sample.json'
combined_data = json.loads(Path(file_path).read_text())

In [20]:
combined_data_test=combined_data[0:10]
for i in combined_data_test:
    print(i['answers'])

['Python']
['Mr. Kneedler']
['Al-Azhar University']
['FARAI CHIDEYA']
['\\epsilon']
['Katherine Marshall Woods']
['baby-shaking case']
['CHIEF JUSTICE ROBERTS']
['10 of 11']
['Don Gonyea']


### The combined data saved to local environment for the later use

## 2 Running the model

### 2.1 Helper Function

In [11]:
from nltk.tokenize import word_tokenize
import collections

def compute_f1(a_gold, a_pred):
    gold_toks = word_tokenize(a_gold)
    pred_toks = word_tokenize(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    
    if num_same == 0:
        return 0
    
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

In [8]:
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 
Also provide me the source for your answer. Explain how to get the answer step by step.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
llm = GPT4All(model="E:/ggml-model-gpt4all-falcon-q4_0.bin", max_tokens=2048)

Found model file at  E:/ggml-model-gpt4all-falcon-q4_0.bin


In [9]:
%%time
answers_pairs=[]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0.1)


for cov in combined_data:
    all_splits = text_splitter.split_text(cov['text'])
    vectorstore = Chroma.from_texts(texts=all_splits, embedding=GPT4AllEmbeddings())
    qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever(), return_source_documents=True)
    
    question = cov['question']
    docs = vectorstore.similarity_search(question)
    predict = qa_chain({"query": question})
    answers_pairs.append((cov['answers'],predict))

Found model file at  C:\\\\Users\\\\Administrator\\\\.cache\\\\gpt4all\\ggml-all-MiniLM-L6-v2-f16.bin
Found model file at  C:\\\\Users\\\\Administrator\\\\.cache\\\\gpt4all\\ggml-all-MiniLM-L6-v2-f16.bin
Found model file at  C:\\\\Users\\\\Administrator\\\\.cache\\\\gpt4all\\ggml-all-MiniLM-L6-v2-f16.bin
Found model file at  C:\\\\Users\\\\Administrator\\\\.cache\\\\gpt4all\\ggml-all-MiniLM-L6-v2-f16.bin
Found model file at  C:\\\\Users\\\\Administrator\\\\.cache\\\\gpt4all\\ggml-all-MiniLM-L6-v2-f16.bin
Found model file at  C:\\\\Users\\\\Administrator\\\\.cache\\\\gpt4all\\ggml-all-MiniLM-L6-v2-f16.bin
Found model file at  C:\\\\Users\\\\Administrator\\\\.cache\\\\gpt4all\\ggml-all-MiniLM-L6-v2-f16.bin
Found model file at  C:\\\\Users\\\\Administrator\\\\.cache\\\\gpt4all\\ggml-all-MiniLM-L6-v2-f16.bin
Found model file at  C:\\\\Users\\\\Administrator\\\\.cache\\\\gpt4all\\ggml-all-MiniLM-L6-v2-f16.bin
Found model file at  C:\\\\Users\\\\Administrator\\\\.cache\\\\gpt4all\\ggml-all-M

### 2.1 Result Evaluation(F1 Score)

In [21]:
for i in answers_pairs:
    f1_score = compute_f1(i[0][0],i[1]['result'])
    print(f"Question: {i[1]['query']}")
    print(f"Predicted Answer: {i[1]['result']}")
    print(f"Actual Answer: {i[0][0]}")
    print(f"F1 Score: {f1_score:.4f}")
    print()

Question: Which program does Suellen reference that shares its name with a snake?
Predicted Answer:  Sueellen references the program called `snake` in her code.
Actual Answer: Python
F1 Score: 0.0000

Question: Which person is talking to the Chief Justice?
Predicted Answer:  The person talking to the Chief Justice is Ms. Lee.
Actual Answer: Mr. Kneedler
F1 Score: 0.0000

Question: What is the name of the most important center of learning of Sunni Islam in the country where the pope is going to celebrate a mass tomorrow ?
Predicted Answer:  The most important center of learning of Sunni Islam in the country where the pope is going to celebrate a mass tomorrow is Al Azhar University.
Actual Answer: Al-Azhar University
F1 Score: 0.0690

Question: Who hosts NEWS & NOTES?
Predicted Answer:  The context does not provide information about who hosts NEWS & NOTEs.
Actual Answer: FARAI CHIDEYA
F1 Score: 0.0000

Question: Which feature in DrRacket inserts an epsilon?
Predicted Answer:  The `equal

## 3 Attempt at Instruct Embedding

In [15]:
!pip install InstructorEmbedding 

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for InstructorEmbedding from https://files.pythonhosted.org/packages/6c/fc/64375441f43cc9ddc81f76a1a8f516e6d63f5b6ecb67fffdcddc0445f0d3/InstructorEmbedding-1.0.1-py2.py3-none-any.whl.metadata
  Using cached InstructorEmbedding-1.0.1-py2.py3-none-any.whl.metadata (20 kB)
Using cached InstructorEmbedding-1.0.1-py2.py3-none-any.whl (19 kB)


In [21]:
pip install torch 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/f9/e6/3c821e7417acd82df89e39f09156ce80d58817b5b4b1ac5453b522bc5dd4/torchvision-0.16.2-cp311-cp311-win_amd64.whl.metadata
  Using cached torchvision-0.16.2-cp311-cp311-win_amd64.whl.metadata (6.6 kB)
  Using cached sentencepiece-0.1.99-cp311-cp311-win_amd64.whl (977 kB)
Using cached torchvision-0.16.2-cp311-cp311-win_amd64.whl (1.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install torch torchvision torchaudio

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/c9/c0/b738db223b85c0096e2c3b1aaa647419f9af68331f8ad09dba6a2d38136c/torchaudio-2.1.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.3 MB 787.7 kB/s eta 0:00:03
   ----- ---------------------------------- 0.3/2.3 MB 2.5 MB/s eta 0:00:01
   ---------------- ----------------------- 1.0/2.3 MB 5.7 MB/s eta 0:00:01
   ---------------------------------------  2.3/2.3 MB 11.3 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 10.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 
Also provide me the source for your answer. Explain how to get the answer step by step.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
llm = GPT4All(model="E:/ggml-model-gpt4all-falcon-q4_0.bin", max_tokens=2048)

Found model file at  E:/ggml-model-gpt4all-falcon-q4_0.bin


In [8]:
instruct_embedding_model_name = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
instruct_embedding_model_kwargs = {'device': 'cpu'}
instruct_embedding_encode_kwargs = {'normalize_embeddings': True}

In [9]:
%%time
answers_pairs=[]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0.1)

word_embed = HuggingFaceInstructEmbeddings(
    model_name=instruct_embedding_model_name,
    model_kwargs=instruct_embedding_model_kwargs,
    encode_kwargs=instruct_embedding_encode_kwargs,
    embed_instruction="Represent the story for retrieval:"
)

for cov in combined_data:
    all_splits = text_splitter.split_text(cov['text'])
    vectorstore = Chroma.from_texts(texts=all_splits, embedding=word_embed)
    qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever(), return_source_documents=True)
    
    question = cov['question']
    docs = vectorstore.similarity_search(question)
    predict = qa_chain({"query": question})
    answers_pairs.append((cov['answers'],predict))

C:\Users\Administrator\AppData\Roaming\Python\Python311\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512
CPU times: total: 1h 25min 42s
Wall time: 21min 18s


In [10]:
for i in answers_pairs:
    f1_score = compute_f1(i[0][0],i[1]['result'])
    print(f"Question: {i[1]['query']}")
    print(f"Predicted Answer: {i[1]['result']}")
    print(f"Actual Answer: {i[0][0]}")
    print(f"F1 Score: {f1_score:.4f}")
    print()

Question: Which program does Suellen reference that shares its name with a snake?
Predicted Answer:  Sueltene does not mention any specific program with the same name as a snake.
Actual Answer: Python
F1 Score: 0.0000

Question: Which person is talking to the Chief Justice?
Predicted Answer:  Ms. Lee is talking to the Chief Justice.
Actual Answer: Mr. Kneedler
F1 Score: 0.0000

Question: What is the name of the most important center of learning of Sunni Islam in the country where the pope is going to celebrate a mass tomorrow ?
Predicted Answer:  The most important center of learning of Sunni Islam in the country where the pope is going to celebrate a mass tomorrow is Al Azhar University.
Actual Answer: Al-Azhar University
F1 Score: 0.0690

Question: Who hosts NEWS & NOTES?
Predicted Answer:  The context does not provide information on who hosts NEWS & NOTEs.
Actual Answer: FARAI CHIDEYA
F1 Score: 0.0000

Question: Which feature in DrRacket inserts an epsilon?
Predicted Answer:  The fe

## 4 Adding question embedding

### 4.1 GPT4ALL

In [6]:
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 
Also provide me the source for your answer. Explain how to get the answer step by step.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
llm = GPT4All(model="E:/ggml-model-gpt4all-falcon-q4_0.bin", max_tokens=2048)

Found model file at  E:/ggml-model-gpt4all-falcon-q4_0.bin


In [7]:
instruct_embedding_model_name = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
instruct_embedding_model_kwargs = {'device': 'cpu'}
instruct_embedding_encode_kwargs = {'normalize_embeddings': True}

In [8]:

answers_pairs=[]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0.1)
model_name = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

def process_story_questions(combined_data, model_name, instruction):
    word_embed = HuggingFaceInstructEmbeddings(
        model_name=instruct_embedding_model_name,
        model_kwargs=instruct_embedding_model_kwargs,
        encode_kwargs=instruct_embedding_encode_kwargs,
        embed_instruction="Represent the story for retrieval:"
    )

    for cov in combined_data:
        all_splits = text_splitter.split_text(cov['text'])
        vectorstore = Chroma.from_texts(texts=all_splits, embedding=word_embed)
        qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever(), return_source_documents=True)

        question = cov['question']
        hf_query_embs = HuggingFaceInstructEmbeddings(
            model_name=instruct_embedding_model_name,
            model_kwargs=instruct_embedding_model_kwargs,
            encode_kwargs=instruct_embedding_encode_kwargs,
            query_instruction=instruction
        )
        question_emb = hf_query_embs.embed_query(question)
        docs = vectorstore.similarity_search_by_vector(question_emb)
        predict = qa_chain({"query": question})
        answers_pairs.append((cov['answers'],predict))
        

In [10]:
%%time
model_name = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
instruction = "Represent the story for retrieval:" 
process_story_questions(combined_data, model_name, instruction)


C:\Users\Administrator\AppData\Roaming\Python\Python311\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_leng

NameError: name 'compute_f1' is not defined

In [12]:
for i in answers_pairs:
    f1_score = compute_f1(i[0][0],i[1]['result'])
    print(f"Question: {i[1]['query']}")
    print(f"Predicted Answer: {i[1]['result']}")
    print(f"Actual Answer: {i[0][0]}")
    print(f"F1 Score: {f1_score:.4f}")
    print()

Question: Which program does Suellen reference that shares its name with a snake?
Predicted Answer:  Sueltene does not mention any specific program with the same name as a snake.
Actual Answer: Python
F1 Score: 0.0000

Question: Which person is talking to the Chief Justice?
Predicted Answer:  Ms. Lee is talking to the Chief Justice.
Actual Answer: Mr. Kneedler
F1 Score: 0.0000

Question: What is the name of the most important center of learning of Sunni Islam in the country where the pope is going to celebrate a mass tomorrow ?
Predicted Answer:  The most important center of learning of Sunni Islam in the country where the pope is going to celebrate a mass tomorrow is Al Azhar University.
Actual Answer: Al-Azhar University
F1 Score: 0.0690

Question: Who hosts NEWS & NOTES?
Predicted Answer:  The context does not provide information on who hosts NEWS & NOTEs.
Actual Answer: FARAI CHIDEYA
F1 Score: 0.0000

Question: Which feature in DrRacket inserts an epsilon?
Predicted Answer:  The fe

### 4.2 Retrieved Sentences

In [ ]:
import nltk
nltk.download('punkt')

In [24]:

from nltk.tokenize import sent_tokenize
answers_pairs=[]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0.1)
model_name = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

def process_story_questions(combined_data, model_name, instruction):
    hf_story_embs = HuggingFaceInstructEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
        embed_instruction="Use the following pieces of context to answer the question at the end:"
    )

    for cov in combined_data:
        sentences = sent_tokenize(cov['text'])
        sentence_embs = hf_story_embs.embed_documents(sentences)
        #all_splits = text_splitter.split_text(cov['text'])
        #vectorstore = Chroma.from_texts(texts=all_splits, embedding=word_embed)
        #qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever(), return_source_documents=True)

        question = cov['question'] 
        hf_query_embs = HuggingFaceInstructEmbeddings(
            model_name=model_name,
            model_kwargs=instruct_embedding_model_kwargs,
            encode_kwargs=instruct_embedding_encode_kwargs,
            query_instruction=instruction
        )
        question_emb = hf_query_embs.embed_query(question)
        scores = [torch.cosine_similarity(torch.tensor(sentence_emb).unsqueeze(0), torch.tensor(question_emb).unsqueeze(0))[0].item() for sentence_emb in sentence_embs]
        
        best_sentence_idx = scores.index(max(scores))
        best_sentence = sentences[best_sentence_idx]
        
        
        #docs = vectorstore.similarity_search_by_vector(question_emb)
        #predict = qa_chain({"query": question})
        answers_pairs.append((cov['question'],cov['answers'],best_sentence))
        
    

In [25]:
model_name = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
instruction = "Represent the story for retrieval:" 
process_story_questions(combined_data, model_name, instruction)

load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_length  512
load INSTRUCTOR_Transformer
max_seq_leng

In [31]:
for i in answers_pairs:
    f1_score = compute_f1(i[1][0],i[2])
    print(f"Question: {i[0]}")
    print(f"Predicted Answer: {i[2]}")
    print(f"Actual Answer: {i[1]}")
    print(f"F1 Score: {f1_score:.4f}")
    print()

Question: Which program does Suellen reference that shares its name with a snake?
Predicted Answer: But I'm sure it can make for quick test development&gt; dislike because of the magicnessyeah, I hate python, where is my `malloc()` ,stuck_out_tongue,Not really the same thingIn this case I'm talking about decorators that magically rewrite things to inject arguments^ that's my biggest concern, too.why magically?
Actual Answer: ['Python']
F1 Score: 0.0000

Question: Which person is talking to the Chief Justice?
Predicted Answer: Chief Justice, and may it please the Court, Much of the work of public employees is performed by speaking or writing, and much of that work concerns matters of public interest.
Actual Answer: ['Mr. Kneedler']
F1 Score: 0.0000

Question: What is the name of the most important center of learning of Sunni Islam in the country where the pope is going to celebrate a mass tomorrow ?
Predicted Answer: It's the most important center of learning of Sunni Islam.
Actual Answ

In [22]:
for i in answers_pairs:
    f1_score = compute_f1(i[0][0],i[1])
    print(f"Question: {i[1]['query']}")
    print(f"Predicted Answer: {i[1]['result']}")
    print(f"Actual Answer: {i[0][0]}")
    print(f"F1 Score: {f1_score:.4f}")
    print()

['Python']
['Mr. Kneedler']
['Al-Azhar University']
['FARAI CHIDEYA']
['\\epsilon']
['Katherine Marshall Woods']
['baby-shaking case']
['CHIEF JUSTICE ROBERTS']
['10 of 11']
['Don Gonyea']
['Leslie.Hansen']
['tech industry']
['Mr. Lamken']
['Federal Whistle-blower Protection Act']
['Section 324']
['six']
['Eighth Amendment']
['May']
['Meritcare v. St. Paul']
['Lehman Brothers Inc']
['the Washington Post']
['`koyo-lib`']
['700']
['were a party to a secret contract or at least a secret relationship']
['a spreading-activation network and a network by which nodes support the continued existence of other nodes']
['company offering product and customer']
['pytest-parallel']
['restrictions on the timing, the frequency, the duration, and the oppressiveness of the search']
['Mr. Deck']
['habeas action']
['the legal due-diligence']
['MR. HURD']
['Iran.']
['struct names']
['Robi Reed']
['Naomi Oreskes']


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def process_story_questions(data, model_name, instruction):
    model_kwargs = {'device': 'cpu'}
    encode_kwargs = {'normalize_embeddings': True}

    # Segment story text into sentences and embed each sentence
    sentences = sent_tokenize(story_text)  # Splitting into sentences
    hf_story_embs = HuggingFaceInstructEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
        embed_instruction="Use the following pieces of context to answer the question at the end:"
    )
    sentence_embs = hf_story_embs.embed_documents(sentences)

    # Process each question
    for question_data in story_data['questions']:
        question = question_data['q']

        hf_query_embs = HuggingFaceInstructEmbeddings(
            model_name=model_name,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs,
            query_instruction=instruction
        )
        question_emb = hf_query_embs.embed_documents([question])[0]

        # Compute cosine similarity scores with each sentence
        scores = [torch.cosine_similarity(torch.tensor(sentence_emb).unsqueeze(0), torch.tensor(question_emb).unsqueeze(0))[0].item() for sentence_emb in sentence_embs]

        best_sentence_idx = scores.index(max(scores))
        best_sentence = sentences[best_sentence_idx]

        # Extract actual answer using the consensus range
        consensus = question_data['consensus']
        actual_answer = story_text[consensus['s']:consensus['e']]
        
        # Calculate F1 score
        f1_score = compute_f1(best_sentence, actual_answer)
        
        # Print results
        print(f"Question: {question}")
        print(f"Predicted Answer: {best_sentence}")
        print(f"Actual Answer: {actual_answer}")
        print(f"F1 Score: {f1_score:.4f}")
        print()
        
        
        # # Calculate F1 score for each sentence
        # f1_scores = [compute_f1(sentence, question) for sentence in sentences]

        # # Find the sentence with the highest F1 score
        # best_sentence_idx = f1_scores.index(max(f1_scores))
        # best_sentence = sentences[best_sentence_idx]

        # # Extract actual answer using the consensus range
        # consensus = question_data['consensus']
        # actual_answer = story_text[consensus['s']:consensus['e']]
        
        # # Calculate F1 score
        # f1_score = compute_f1(best_sentence, actual_answer)
        
        # # Print results
        # print(f"Question: {question}")
        # print(f"Predicted Answer: {best_sentence}")
        # print(f"Actual Answer: {actual_answer}")
        # print(f"F1 Score: {f1_score:.4f}")
        # print()

In [ ]:
model_name = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
instruction = "Represent the story for retrieval:" 
process_story_questions(combined_data, model_name, instruction)